`pidgin.doctesting` provides tooling to apply input transformations to doctest strings.

In [1]:
    import ast, doctest, IPython, textwrap, re, collections, importnb

In [2]:
    class OutputChecker(doctest.OutputChecker):
        def check_output(self, want, got, optionflags):
            return True if want == '...\n' else super().check_output(want, got, optionflags)

In [3]:
    def run_docstring_examples(
        f, ip, verbose=False, compileflags=None, optionflags=0
    ):
        finder = doctest.DocTestFinder(verbose=verbose)
        runner = doctest.DocTestRunner(verbose=verbose, optionflags=optionflags, checker=OutputChecker())
        for test in (doctest_to_ipython(ip, object) for object in finder.find(
            f, ip.user_ns['__name__']
        )):
            test.globs = ip.user_ns
            runner.run(test, compileflags=compileflags, clear_globs=False)        
                
        return runner

In [4]:
    def doctest_to_ipython(ip, test):
        for example in test.examples:
            example.source = ip.input_transformer_manager.transform_cell(
                textwrap.indent(example.source, ' '*4)
            )
            if '```' in example.want:
                example.want, ticks, rest = example.want.rpartition('```')
        return test

`run_cell_doctests` runs any `doctest`s found in the raw source.

In [5]:
    def run_cell_doctests(ip, result) -> Exception:
        with IPython.utils.capture.capture_output(
            stdout=True, stderr=False, display=False
        ) as output:
            run_docstring_examples(result.info.raw_cell, ip)

        if output.stdout:
            return DocTestException(output.stdout)

In [6]:
    class DocTestException(BaseException):
        def _render_traceback_(self): return args[0].splitlines()

In [7]:
    def load_ipython_extension(ip):
        ip.input_transformer_manager.cleanup_transforms = [
            object for object in ip.input_transformer_manager.cleanup_transforms
            if object is not IPython.core.inputtransformer2.classic_prompt
        ]
        
    def unload_ipython_extension(ip): ...
        
    if __name__ == '__main__': 
        %reload_ext pidgin.tangle
        %reload_ext pidgin.display
        unload_ipython_extension(IPython.get_ipython())
        load_ipython_extension(IPython.get_ipython())
